In [1]:
# -*- coding: utf-8 -*-
from hanspell import spell_checker
from UrlParserForKBO import UrlParserForKBO
from Parser_DaumKBO import Parser_DaumKBO
from Parser_KBO import Parser_KBO
from sklearn import tree
from sklearn.externals.six import StringIO
from getInitialTeamName import getInitialTeamName

import pandas as pd
import numpy as np
import MySQLdb
import random
import datetime
import os
import sys

ImportError: No module named getInitialTeamName

In [ ]:
# DecisionTree - 서두
path_INTRO = './DecisionLearning//DecisionTree_sample.csv'
df_INTRO = pd.read_csv(path_INTRO,index_col=None,header=None,names=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum','context'])
score_INTRO = np.array(df_INTRO.reindex(columns=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum']))
context_INTRO = np.array(df_INTRO.get('context'))
context_sen_INTRO = [' ','승리하였다. ','접전끝에 승리를 하였다. ','역전에 성공했다. ','완승을 하였다. ','무승부로 끝이났다. ','영봉승을 하였다. ']
clf_INTRO = tree.DecisionTreeClassifier(criterion='entropy')
clf_INTRO = clf_INTRO.fit(score_INTRO,context_INTRO)

In [ ]:
def getClassifier():
    return clf_INTRO.predict([x-y for x,y in zip(Parser_KBO.h_score[:13],Parser_KBO.a_score[:13])])[0]

In [ ]:
def writeDate(parser_KBO):    
    return Parser_KBO.date

In [ ]:
def writeHead(parser_KBO,parser_DaumKBO):

    # HEAD
    HEAD=u'at ascore vs ht hscore'
    HEAD=HEAD.replace('at',Parser_KBO.a_team)
    HEAD=HEAD.replace('ht',Parser_KBO.h_team)
    HEAD=HEAD.replace('hscore',str(Parser_KBO.h_score[12]))
    HEAD=HEAD.replace('ascore',str(Parser_KBO.a_score[12]))
    
    return HEAD

In [ ]:
def writeIntro(Parser_KBO,Parser_DaumKBO,contextClassifier):
    MVP_PITCHER=None
    
    sentences1=(
        u'winTeam는 date stadium 경기장에서 열린 loseTeam와의 경기에서 ascore-hscore 으로 context',
        u'date stadium에서 열린 at와 ht와의 경기에서 ascore-hscore로 winTeam이 context',
        u'at와 ht의 경기가 date stadium에서 있었던 오늘 at는 ascore, ht는 hscore로 winTeam가 경기에서 context',
        u'date stadium에서 있었던 at와 ht와의 경기, winTeam이 winScore loseTeam이 loseScore로 winTeam이 context',
        u'홈팀 ht와 어웨이팀 at간의 경기가 오늘 date 펼쳐졌다. stadium경기장에서 열린 이 경기에서 winTeam이 winScore : loseScore로 loseTeam을 context',
    )
    sentences2=(
        u'이날 경기에서 mvp_pitcher선수가 투구수NP 피안타H 의 기록을 보여주며 호투를 하였다. ',
        u'이번 경기에서 winTeam의  mvp_pitcher선수가 INN이닝을 호투하며, R실점 SO삼진을 잡아내며 승리를 이끌었다. ',
        u'오늘 경기간에 mvp_pitcher선수가 INN이닝을 던지고 SO삼진 ER자책 R실점 H피안타를 기록하여 팀의 승리에 기여하였다. '
    )
    
    # INTRO1
    INTRO1=sentences1[random.randint(0,len(sentences1)-1)]
    INTRO1=INTRO1.replace('winTeam',Parser_KBO.winTeam)
    INTRO1=INTRO1.replace('loseTeam',Parser_KBO.loseTeam)
    INTRO1=INTRO1.replace('winScore',str(Parser_KBO.winScore))
    INTRO1=INTRO1.replace('loseScore',str(Parser_KBO.loseScore))
    INTRO1=INTRO1.replace('stadium',Parser_KBO.stadium)
    INTRO1=INTRO1.replace('hscore',str(Parser_KBO.h_score[12]))
    INTRO1=INTRO1.replace('ascore',str(Parser_KBO.a_score[12]))
    INTRO1=INTRO1.replace('date',str(int(Parser_KBO.date[4:6]))+u'월'+str(int(Parser_KBO.date[6:8]))+u'일'+Parser_KBO.date[8:])
    INTRO1=INTRO1.replace('ht',Parser_KBO.h_team)
    INTRO1=INTRO1.replace('at',Parser_KBO.a_team)    
    
    # INTRO-context
    #'context' -> exception_case(무승부)
    if Parser_KBO.h_score[12]==Parser_KBO.a_score[12]:
        INTRO1=INTRO1.replace('context',context_sen_INTRO[5].decode('utf-8'))    
    #'context' -> exception_case(영봉승)
    elif Parser_KBO.h_score[12]==0 or Parser_KBO.a_score[12]==0:
        INTRO1=INTRO1.replace('context',context_sen_INTRO[6].decode('utf-8'))    
    #'context'-> 나머지의 경우
    else:
        INTRO1=INTRO1.replace('context',context_sen_INTRO[getClassifier(url)].decode('utf-8'))
    
    # MVP_PITCHER
    for row in range(len(Parser_KBO.winTeam_pitRecord)):
        ERA=Parser_KBO.winTeam_pitRecord[u'평균자책점'][row]
        if type(ERA)==float:
            ERA=float(ERA)
        else:
            ERA=0
        BN=int(Parser_KBO.winTeam_pitRecord[u'타자'][row])
        H=int(Parser_KBO.winTeam_pitRecord[u'피안타'][row])
        SO=int(Parser_KBO.winTeam_pitRecord[u'삼진'][row])
        arr=np.array([ERA,BN,H,SO])
                       
        INTRO2=''
        if clf_MVP_PITCHER.predict(arr)[0]:
            MVP_PITCHER=Parser_KBO.winTeam_pitRecord[u'선수명'][row] 
            #INTRO2
            INTRO2=sentences2[random.randint(0,len(sentences2)-1)]
            INTRO2=INTRO2.replace('winTeam',Parser_KBO.winTeam)
            INTRO2=INTRO2.replace('mvp_pitcher',Parser_KBO.winTeam_pitRecord.iloc[row][u'선수명'])
            INTRO2=INTRO2.replace('ERA',Parser_KBO.winTeam_pitRecord.iloc[row][u'평균자책점'])
            INTRO2=INTRO2.replace('INN',Parser_KBO.winTeam_pitRecord.iloc[row][u'이닝'])
            INTRO2=INTRO2.replace('BN',Parser_KBO.winTeam_pitRecord.iloc[row][u'타자'])
            INTRO2=INTRO2.replace('NP',Parser_KBO.winTeam_pitRecord.iloc[row][u'투구수'])
            INTRO2=INTRO2.replace('SO',Parser_KBO.winTeam_pitRecord.iloc[row][u'삼진'])            
            INTRO2=INTRO2.replace('ER',Parser_KBO.winTeam_pitRecord.iloc[row][u'자책'])
            INTRO2=INTRO2.replace('R',Parser_KBO.winTeam_pitRecord.iloc[row][u'실점'])
            INTRO2=INTRO2.replace('H',Parser_KBO.winTeam_pitRecord.iloc[row][u'피안타'])
            break
            
    return INTRO1+INTRO2

In [2]:
# 결론작성
def writeConc(parser_KBO,parser_DaumKBO):
    # Conc
    Conclusion=Parser_KBO.a_status   
    
    return Conclusion

In [3]:
def getEmblem(parser_KBO):
    emblem=getInitialTeamName.get(Parser_KBO.winTeam)
    return emblem

In [4]:
# 본문작성
def writeMain(parser_KBO,parser_DaumKBO):
    url=url.replace('BoxScore','Situation')
    month=Parser_KBO.date[4:6]
    day=Parser_KBO.date[6:8]
    critical_Inning=getCriticalInning(date='2016'+month+day,awayTeam=unicode(Parser_KBO.a_team))
#     MAIN1 : about critical Inning
    MAIN1=u''
#     MAIN2 : about the flow of critical Inning
    MAIN2=u''
#     MAIN3 : about the keyPlayer(batter)
    MAIN3=u''
    
    
#     MAIN1 BEGIN
#     no critical Inning
    if len(critical_Inning)==0:
        sen=(
            u'',
            u'결정적인 이닝이 없던 무난한 경기였다. ',
            u'어느 이닝이 중요하다 뽑을수가 없는 경기였다'
            )
        sen=sen[random.randint(0,len(sen)-1)]
        
#         1 or more critical Exist
    else:
        inning=','.join([inn[0] for inn in critical_Inning])
        sen=(
            u'경기의 양상은 inning에서 판가름났다. ',
            u'승부는 inning이닝에서 갈리게 되었다. ',
            u'승부수는 바로 inning에서 갈라지게 되었다. ',
            u'양팀간의 승부는 inning에서 뚜렷하게 나타났다. ',
            u'두팀의 승부는 결국 inning에서 두드러지게 되었다. '
                )
        sen=sen[random.randint(0,len(sen)-1)]
        sen=sen.replace('inning',inning)
#         MAIN1 END
    MAIN1+=sen

#     MAIN2 BEGIN
    for inn in critical_Inning:
        MAIN2=u'inn 이닝에서 '
        MAIN2=MAIN2.replace(u'inn',inn[0])
#         open All Of Flow
        data=urllib2.urlopen(url+'#sms11')
        html=BeautifulSoup(data)
        cri=str.format('div.situation div#sms%02d > table'%(int(inn[0])))
#                              first = AWAY WIN
        if inn[1]=='first':
            cri=html.select(cri)[0]
#                              second= HOME WIN
        else:
            cri=html.select(cri)[1]
        
        hitContextList=[u'안타',u'루타',u'홈런']
        tmphitList=[]
        hitList=[]
        for line in cri.text.splitlines():
            line = line.strip()
            if line!='':
                tmphitList.append(line)
        for sen in tmphitList[2:]:
            hitList.append(sen.split(' ',2)[2])
        
        for sen in hitList:
#             print sen
            for context in hitContextList:
#                 print context
                if context in sen:
                    player=unicode(sen.split(' ',1)[0])
                    action=unicode(sen.split(' ',1)[1])
                    ex_sen=u'player선수의 action,'
                    ex_sen=ex_sen.replace(u'player',player) 
                    ex_sen=ex_sen.replace(u'action',action)
                    MAIN2+=ex_sen
                    break
                            
                    
        MAIN2+=u'등이 Parser_KBO.winTeam의 승리에 큰 기여를 하였다. '
        MAIN2=MAIN2.replace(u'Parser_KBO.winTeam',Parser_KBO.winTeam)
#         MAIN2 END

#         MAIN3 BEGIN
    MAIN3=u''
    year=Parser_KBO.date[:4]
    month=Parser_KBO.date[4:6]
    day=Parser_KBO.date[6:8]
    url='http://score.sports.media.daum.net/schedule/baseball/kbo/main.daum?game_year=%s&game_month=%s'%(year,month)
    page=urllib2.urlopen(url)
    html=BeautifulSoup(page)
    
    s=unicode(html)
    date_index=s.find(str.format('<td class="time_date" rowspan="5">%d<span class="txt_day">')%(int(day)))
    s=s[date_index:]
    
    s=s[s.find(Parser_KBO.a_team):]
    serial_find_url='http://sports.media.daum.net/sports/gamecenter/'
    serial_start_index=s.find(serial_find_url)+len(serial_find_url)
    serial=s[serial_start_index:serial_start_index+8]
    
    url=str.format('http://m.sports.media.daum.net/m/sports/pack/3min/%s?lineup')%(serial)
    
    if os.name=='nt':
        driver=webdriver.PhantomJS(executable_path='./phantomjs.exe')
    else:
        driver=webdriver.PhantomJS(executable_path='./phantomjs')
    driver.get(url)
    html=driver.page_source
#         PhantomJS 드라이버 닫기
    driver.close()
#         Window에서는 PhantomJS프로세스가 남아있으므로 강제종료
    if os.name=='nt':
        os.system('taskkill /f /im phantomjs.exe')
    soup=BeautifulSoup(html)
    
    keyPlayerInfo=[]
    for team,css_select in zip((u'AWAY',u'HOME'),('td.position.away.key-player','td.position.home.key-player')):
        if soup.select_one(css_select)==None:
            continue
        sel=soup.select_one(css_select)
        info={}
        info[u'선수명']=sel.findNextSibling().text
        info[u'홈어웨이']=team
        
        if team==u'AWAY':
            keyPlayerBattingInfo_toDict=Parser_KBO.a_batRecord[Parser_KBO.a_batRecord[u'선수명']==info[u'선수명']]
            keyPlayerBattingInfo_toDict=keyPlayerBattingInfo_toDict.to_dict('records')[0]
        else:
            keyPlayerBattingInfo_toDict=Parser_KBO.h_batRecord[Parser_KBO.h_batRecord[u'선수명']==info[u'선수명']]
            keyPlayerBattingInfo_toDict=keyPlayerBattingInfo_toDict.to_dict('records')[0]
        
        info[u'득점']=keyPlayerBattingInfo_toDict[u'득점']
        info[u'타율']=keyPlayerBattingInfo_toDict[u'타율']
        info[u'타점']=keyPlayerBattingInfo_toDict[u'타점']
        info[u'타수']=keyPlayerBattingInfo_toDict[u'타수']
        info[u'안타']=keyPlayerBattingInfo_toDict[u'안타']
        keyPlayerInfo.append(info)
    
    for kPlayer in keyPlayerInfo:
    #     승리한팀에 키플레이어가 존재하는 경우
        if kPlayer[u'선수명'] in Parser_KBO.winTeam_batRecord[u'선수명'].values:
            MAIN3=(
                u'Parser_KBO.winTeam의 kPlayer 선수가 Movement 를 기록하여 팀의 키플레이어 역할을 해주었다. ',
                u'Parser_KBO.winTeam kPlayer 선수가 Movement 를 기록 핵심타자 역할을 해주었다. ',
                u'kPlayer선수가 Movement를 하며, Parser_KBO.winTeam의 핵심적인 득점원이 되어주었다. '
            )
            MAIN3=MAIN3[random.randint(0,len(MAIN3)-1)]
            MAIN3=MAIN3.replace(u'Parser_KBO.winTeam',Parser_KBO.winTeam)
            MAIN3=MAIN3.replace(u'kPlayer',kPlayer[u'선수명'])
            Movement=u'Deukjum Tayul Tajum Tasu Anta'
            Movement=Movement.replace(u'Deukjum',u'득점 '+kPlayer[u'득점'])
            Movement=Movement.replace(u'Tayul',u'타율 '+kPlayer[u'타율'])
            Movement=Movement.replace(u'Tajum',u'타점 '+kPlayer[u'타점'])
            Movement=Movement.replace(u'Tasu',u'타수 '+kPlayer[u'타수'])
            Movement=Movement.replace(u'Anta',u'안타 '+kPlayer[u'안타'])
            MAIN3=MAIN3.replace(u'Movement',Movement)

    #     패배한팀에 키플레이어가 존재하는 경우
        else:
            MAIN3=(
                u'',
                u'Parser_KBO.loseTeam의 kPlayer선수가 Movement 를 하였지만 아쉽게도 팀을 승리로 이끌기에는 부족하였다. ',
                u'kPlayer선수 또한 Movement를 하였지만 Parser_KBO.loseTeam의 승리를 돕기에는 부족했다. ',
                u'Parser_KBO.loseTeam의 kPlayer선수가 Movement 를 하며 열심히 방망이질을 했지만 아쉬운 결과였다. '
                )
            MAIN3=MAIN3[random.randint(0,len(MAIN3)-1)]
            MAIN3=MAIN3.replace(u'Parser_KBO.loseTeam',Parser_KBO.loseTeam)
            MAIN3=MAIN3.replace(u'kPlayer',kPlayer[u'선수명'])
            Movement=u'Deukjum Tayul Tajum Tasu Anta'
            Movement=Movement.replace(u'Deukjum',u'득점 '+kPlayer[u'득점'])
            Movement=Movement.replace(u'Tayul',u'타율 '+kPlayer[u'타율'])
            Movement=Movement.replace(u'Tajum',u'타점 '+kPlayer[u'타점'])
            Movement=Movement.replace(u'Tasu',u'타수 '+kPlayer[u'타수'])
            Movement=Movement.replace(u'Anta',u'안타 '+kPlayer[u'안타'])
            MAIN3=MAIN3.replace(u'Movement',Movement)
     
    return MAIN1+MAIN2+MAIN3

In [5]:
startDate='20160824'
endDate='20160824'
urlParserForKBO = UrlParserForKBO(startDate,endDate)

In [12]:
# MySQL conf
con=MySQLdb.connect(host='218.150.181.131',user='root',passwd='1234',db='link10th',charset='utf8', use_unicode=True)

for url in urlParserForKBO.urlList:
    parser_KBO=Parser_KBO(url)
    parser_DaumKBO=Parser_DaumKBO(parser_KBO.boxScore['date'],parser_KBO.boxScore['awayTeam'])
    try:
        date=writeDate(parser_KBO)
        contextClassifier=getClassifier()
        Head=writeHead(parser_KBO,parser_DaumKBO)
        Intro=writeIntro(parser_KBO,parser_DaumKBO,contextClassifier)
        Main=writeMain(parser_KBO,parser_DaumKBO)
        Conc=writeConc(parser_KBO,parser_DaumKBO)
        emblem=getEmblem(parser_KBO)
    except:
        sys.stderr.write(unicode.format(u'********** 날짜 : %s %s와 %s의 경기 파싱오류 ********** '%(Parser_KBO.date,Parser_KBO.a_team,Parser_KBO.h_team)))
    
    #교정부분
#     Head = spell_checker.check(Head)
#     Intro = spell_checker.check(Intro)
#     Main = spell_checker.check(Main)
#     Conc = spell_checker.check(Conc)
#     Head = Head.checked
#     Intro = Intro.checked
#     Main = Main.checked
#     Conc = Conc.checked
     
    cursor=con.cursor()
    cursor.execute(
    '''
    INSERT INTO Article(date,Head,Intro,Main,url,emblem,winCount,loseCount,drawCount)
    VALUES (\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\'),\'%s\'),\'%s\'))
    '''
    %(date,Head,Intro,Main,url,emblem,winCount,loseCount,drawCount)

    )
    # MySQL disconnect
    con.commit()
    cursor.close()
con.close()

C:\Users\GoHyunyoung98\AppData\Local\Enthought\Canopy\User\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
C:\Users\GoHyunyoung98\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\GoHyunyoung98\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\utils\valida